In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
# from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import root_mean_squared_error
import pickle
SAMPLE_FRAC = 0.3

In [2]:
# Prefer pyarrow for all parquet IO in pandas
import pandas as pd
pd.options.io.parquet.engine = 'pyarrow'  # affects read_parquet/to_parquet defaults


In [3]:
!python --version

Python 3.12.12


In [4]:
pd.__version__

'2.3.1'

In [5]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

print(mlflow.get_experiment_by_name("nyc-taxi-experiment"))

2025/10/18 16:09:59 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/10/18 16:09:59 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2025/10/18 16:09:59 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1760793866099, experiment_id='1', last_update_time=1760793866099, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>


In [8]:
# def read_dataframe(filename):
#     df = pd.read_parquet(filename, engine='pyarrow')
    
#     df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
#     df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
#     df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
#     df.duration = df.duration.dt.total_seconds() / 60

#     df = df[(df.duration >= 1) & (df.duration <= 60)]
    
#     categorical = ['PULocationID', 'DOLocationID']

#     df[categorical] = df[categorical].astype(str)
    
#     return df

def read_dataframe(filename: str, sample_frac: float | None = None) -> pd.DataFrame:
    # Read only necessary columns to reduce memory
    cols = [
        'lpep_dropoff_datetime', 'lpep_pickup_datetime',
        'PULocationID', 'DOLocationID', 'trip_distance'
    ]
    print(f"Reading parquet from {filename} with pyarrow and columns={cols} ...", flush=True)
    df = pd.read_parquet(filename, engine='pyarrow', columns=cols)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    if sample_frac is not None and 0 < sample_frac < 1:
        df = df.sample(frac=sample_frac, random_state=42).reset_index(drop=True)
        print(f"Sampled dataframe to frac={sample_frac}: shape={df.shape}")
    else:
        print(f"Loaded dataframe shape: {df.shape}")
    return df

In [9]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet', sample_frac = SAMPLE_FRAC)
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet', sample_frac = SAMPLE_FRAC)

Reading parquet from https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet with pyarrow and columns=['lpep_dropoff_datetime', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance'] ...
Sampled dataframe to frac=0.3: shape=(22172, 6)
Reading parquet from https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet with pyarrow and columns=['lpep_dropoff_datetime', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance'] ...
Sampled dataframe to frac=0.3: shape=(22172, 6)
Reading parquet from https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet with pyarrow and columns=['lpep_dropoff_datetime', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance'] ...
Sampled dataframe to frac=0.3: shape=(18576, 6)
Sampled dataframe to frac=0.3: shape=(18576, 6)


In [10]:
len(df_train), len(df_val)

(22172, 18576)

In [11]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [12]:
categorical = ['PU_DO'] 
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [13]:
target = 'duration'
y_train = df_train[target].values 
y_val = df_val[target].values

In [ ]:
# lr = LinearRegression()
# lr.fit(X_train, y_train)

# y_pred = lr.predict(X_train)

# root_mean_squared_error(y_train, y_pred)

np.float64(5.699564118198954)

In [ ]:
# with open('models/lin_reg.bin', 'wb') as f_out:
#     pickle.dump((dv, lr), f_out)

In [ ]:
# with mlflow.start_run():

#     mlflow.set_tag("developer", "Kristof")
#     mlflow.log_param("train-data-path", 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
#     mlflow.log_param("valid-data-path", 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')

#     alpha = 0.01
#     mlflow.log_param("alpha", alpha)
#     lr = Lasso(alpha=alpha)
#     lr.fit(X_train, y_train)

#     y_pred = lr.predict(X_val)
#     rmse = root_mean_squared_error(y_val, y_pred)

#     mlflow.log_metric("rmse", rmse)

#     mlflow.log_artifact('models/lin_reg.bin', artifact_path='models_pickle')


In [ ]:
# # Example: write a sample parquet with pyarrow and log via MLflow
# sample_path = 'models/sample_data.parquet'
# df.head(1000).to_parquet(sample_path, engine='pyarrow', index=False)
# mlflow.log_artifact(sample_path, artifact_path='datasets')


## XGboost

In [15]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

/opt/conda/envs/conda-env/lib/python3.12/site-packages/hyperopt/atpe.py:19: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [16]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [ ]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)

        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )

        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)
        
    return {'loss': rmse, 'status': STATUS_OK}

In [ ]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
    }

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials())

In [29]:
mlflow.xgboost.autolog(False)

In [ ]:
with mlflow.start_run():
    best_params = {
        'max_depth': 48,
        'learning_rate': 0.1256073328183555,
        'reg_alpha': 0.34485337886791795,
        'reg_lambda': 0.018950580168387268,
        'min_child_weight': 1.492447195778266,
        'objective': 'reg:linear',
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=100,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    with open('models/preprocessor.b', 'wb') as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    # Log the XGBoost model with signature
    signature = mlflow.models.infer_signature(X_val, y_pred)
    
    mlflow.xgboost.log_model(
        xgb_model=booster, 
        name="MyModel",
        signature=signature
    )


In [14]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog(
    log_models=True,
    log_datasets=True,
    log_input_examples=True,
    log_model_signatures=True,
)

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

/home/codespace/.local/lib/python3.12/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
